<a href="https://colab.research.google.com/github/Mahesh18190/mahesh/blob/main/Resume_Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q\
  langchain==0.1.16\
  langchain-community==0.0.34\
  langchain-core==0.1.45\
  chromadb \
  pypdf \
  gradio \
  langchain-google-genai \
  google-genai

In [5]:
from langchain import embeddings
!pip install -q sentence-transformers

# import the necessary packages

import os
import gradio as gr
from google.colab import userdata


from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI


#HuggingFaace Embeddings

from langchain.embeddings import HuggingFaceEmbeddings


#Agent

from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool



#API KEY

os.environ["GOOGLE_API_KEY"]=userdata.get("GOOGLE")

#LLM + Embeddings

llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.2
)

#Hf Embeddings

embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Vectordb= None

#Process PDF's

def process_pdf(resume_file,jd_file):
  global Vectordb


  if resume_file is None or jd_file is None:
    return "Upload Both Resume and Job Description PDFS"


  splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
  docs=[]
 #Resume

  resume_docs = PyPDFLoader(resume_file.name).load()

  for d in resume_docs:
    d.metadata["type"]="RESUME"
    d.metadata["source"]=resume_file.name
  docs.extend(splitter.split_documents(resume_docs))


  #JD

  jd_docs = PyPDFLoader(jd_file.name).load()
  for d in jd_docs:
    d.metadata["type"]="JD"
    d.metadata["sorce"]=jd_file.name
  docs.extend(splitter.split_documents(jd_docs))


  Vectordb= Chroma.from_documents(
      documents=docs,
      embedding=embeddings
  )

  return f"Resume  & JD processed.Total chunks:{len(docs)}.Now go to 'Ask' tab"


  #tool
@tool
def search_docs(query:str) ->str:
  """Search resume and Jd content and return evidence."""
  global Vectordb
  if Vectordb is None:
    return "NO DOCUMENTS:please upload and process PDFs first"
  results=Vectordb.as_retriever(search_kwargs={"k":6}).invoke(query)
  if not results:
    return "NO MATCHES:nothing relevant found in the documents"

  return "\n\n---\n\n".join(
      f"[{r.metadata.get('type')}] {r.page_content}"
      for r in results
  )


  #Agent
SYSTEM_RULES="""
  You are a resume fit scoring agent.

  TASK:
  -Compare resume VS Job descripption
  -Give fir score(0-100)
  -Verdict(Strong/Moderate/Weak)
  -Strengths
  -Missings Skills/Gaps
  -Resume Improvements
  -Interview prep topics

  RULES:
  -Use Search_docs before answering (collect evidence).
  -Do not halluciate skills/experience not present in the resume .
  -if info is missing ,say: "I don't see this information in the documents."
  -Keep the output structured and simple .
  """

agent= initialize_agent(
    tools=[search_docs],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)


#chat

def resume_chat(message,history):
  return agent.run(f"{SYSTEM_RULES}\n\nUser question :{message}")


#UI

with gr.Blocks() as demo:
  gr.Markdown("# Resume Fit Scorer")

  with gr.Tab("Upload"):
    r= gr.File(label="Resume PDF",file_types=["pdf"])
    j= gr.File(label="Job Description PDF",file_types=["pdf"])
    b= gr.Button("Process",variant="primary")
    s= gr.Textbox(label="Status", lines=5,interactive=False)
    b.click(process_pdf,[r,j],s)

  with gr.Tab("Ask"):
    chatbot = gr.Chatbot(label="Chat History", height=400)
    msg = gr.Textbox(label="Your message", placeholder="Type your message here...")
    send_btn = gr.Button("Send")

    def chat_function_manual(message, history):
        if message.strip() == "":
            return "", history

        response = agent.run(f"{SYSTEM_RULES}\n\nUser question :{message}")
        history.append([message, response])
        return "", history

    msg.submit(chat_function_manual, [msg, chatbot], [msg, chatbot])
    send_btn.click(chat_function_manual, [msg, chatbot], [msg, chatbot])

    gr.Examples(
        examples=[
            "Give me fit score (0-100) and missing skills.",
            "List strength and gaps for this JD.",
            "How can i improve my resume for this role?.",
            "What interview topics should i prepare based on missing skills?."
        ],
        inputs=[msg],
        outputs=[msg]
    )

demo.launch()

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
/tmp/ipython-input-3959420068.py:149: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat History", height=400)
/tmp/ipython-input-3959420068.py:149: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(label="Cha

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe10359eec45c47073.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
